In [ ]:
!pip install cloud-tpu-client==0.10 torch==1.13.0 torchvision==0.13.0 https://storage.googleapis.com/tpu-pytorch/wheels/colab/torch_xla-1.13-cp38-cp38-linux_x86_64.whl

In [20]:
!pip install torch_xla

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [26]:
!pip install tensorflow-neuronx

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
!pip install pytorch-lightning bert_score datasets rouge_score evaluate transformers wandb accelerate

In [3]:
!git clone https://github.com/Myashka/CQA_RLHF.git

Cloning into 'CQA_RLHF'...
remote: Enumerating objects: 177, done.
remote: Counting objects: 100% (177/177), done.
remote: Compressing objects: 100% (115/115), done.
remote: Total 177 (delta 113), reused 119 (delta 58), pack-reused 0
Receiving objects: 100% (177/177), 55.48 KiB | 2.22 MiB/s, done.
Resolving deltas: 100% (113/113), done.


In [4]:
if 'google.colab' in str(get_ipython()):
    from google.colab import drive
    drive.mount('/content/drive')

Mounted at /content/drive


In [5]:
import yaml

In [24]:
trainer_config = dict(
    seed=42,
    model_name="EleutherAI/gpt-neo-125M",
    wandb=dict(
        api="60fce56bfaec85b8d6bc78bfac2086891f5afe54",
        project_name="CQA_RLHF",
        args=dict(job_type="train", group="sft", name="test-run"),
    ),
    data=dict(
        path_to_data=r"/content/drive/MyDrive/Diploma/data/1.0-data-div-ans-sep.json",
        max_length=512,
        batch_size=2,
    ),
    model_params=dict(
        lr=5e-5,
        do_freeze=True,
        use_cache=False,
        warmup_steps=100,
        adam_betas=[0.9, 0.95],
        weight_decay=0.001,
        do_compute_metrics = False
    ),
    trainer=dict(
        checkpoint=dict(
            every_n_train_steps=2000,
            dirpath=r"/content/drive/MyDrive/Diploma/Checkpoints",
        ),
        params=dict(
            accelerator="tpu",
            max_epochs=3,
            auto_scale_batch_size=True,
            accumulate_grad_batches=1,
            gradient_clip_val=1,
            precision="16",
            ckpt_path=None,
            val_check_interval=1000,
            overfit_batches=0,  # 0 for train
            num_sanity_val_steps=2,
        ),
    ),
)

with open("trainer_config.yaml", "w") as outfile:
    yaml.dump(trainer_config, outfile, default_flow_style=False)

In [27]:
!python /content/CQA_RLHF/sft/lightning/pl_train.py --config_file /content/trainer_config.yaml

2023-02-22 18:31:01.828354: I tensorflow/core/platform/cpu_feature_guard.cc:193] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-02-22 18:31:03.078716: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer.so.7'; dlerror: libnvinfer.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-22 18:31:03.078837: W tensorflow/compiler/xla/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libnvinfer_plugin.so.7'; dlerror: libnvinfer_plugin.so.7: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /usr/local/nvidia/lib:/usr/local/nvidia/lib64
2023-02-22 18:31:03.078855: W tensorfl

In [ ]:
!python /content/CQA_RLHF/sft/lightning/pl_train.py --config_file /content/trainer_config.yaml